In [1]:
import pandas as pd
import numpy as np

In [2]:
products = pd.read_csv("C:\\Users\\dell\\Downloads\\EcopackAI\\data\\products_cleaned.csv")
materials = pd.read_csv("C:\\Users\\dell\\Downloads\\EcopackAI\\data\\materials_cleaned.csv")

In [3]:
print(products.shape)
print(materials.shape)

products.head()
materials.head()

(134, 12)
(111, 8)


,material_id,material_name,material_type,strength_score,weight_cap,biodegradability,co2_emission,recyclability_percent
0,1,Recycled Cardboard / Paperboard,Fiber-based (cellulosic),3,3,85,70,75
1,2,Molded Pulp (Recycled Paper Fiber),Fiber-based (cellulosic),3,3,85,70,70
2,3,"Kraft Paper Wraps, Paper Padding",Fiber-based,2,2,90,70,70
3,4,Starch-Based Foams/Peanuts,Bio-based polymer (starch),2,3,95,85,30
4,5,Mycelium-Based Molded Packaging,Bio-based composite,3,3,95,90,20


In [4]:
products.columns = products.columns.str.lower().str.strip()
materials.columns = materials.columns.str.lower().str.strip()

In [6]:
print(products.columns)
print(materials.columns)

Index(['product_id', 'product_name', 'sector', 'main_packaging_material',
       'strength_score', 'weight_capacity_score', 'barrier_score',
       'biodegradability_score', 'co2_emission_score', 'recyclability_percent',
       'cost_score', 'reuse_potential_score'],
      dtype='object')
Index(['material_id', 'material_name', 'material_type', 'strength_score',
       'weight_cap', 'biodegradability', 'co2_emission',
       'recyclability_percent'],
      dtype='object')


In [ ]:
products["main_packaging_material"] = (
    products["main_packaging_material"].str.lower().str.strip()
)

materials["material_type"] = (
    materials["material_type"].str.lower().str.strip()
)



In [15]:
products["key"] = 1
materials["key"] = 1

merged = pd.merge(products, materials, on="key").drop("key", axis=1)
print(merged.shape)


(14874, 20)


In [17]:
print(merged.columns)


Index(['product_id', 'product_name', 'sector', 'main_packaging_material',
       'strength_score_x', 'weight_capacity_score', 'barrier_score',
       'biodegradability_score', 'co2_emission_score',
       'recyclability_percent_x', 'cost_score', 'reuse_potential_score',
       'material_id', 'material_name', 'material_type', 'strength_score_y',
       'weight_cap', 'biodegradability', 'co2_emission',
       'recyclability_percent_y'],
      dtype='object')


In [18]:
# strength difference (material - product)
merged["strength_gap"] = merged["strength_score_y"] - merged["strength_score_x"]

# weight difference (material capacity - product requirement)
merged["weight_gap"] = merged["weight_cap"] - merged["weight_capacity_score"]


In [19]:
merged[["strength_gap", "weight_gap"]].head()
print(merged.shape)


(14874, 22)


In [20]:
merged.drop(
    columns=["material_type"],
    inplace=True
)


In [21]:
merged.rename(
    columns={
        "strength_score_x": "product_strength_req",
        "strength_score_y": "material_strength"
    },
    inplace=True
)


In [22]:
print(merged.shape)
merged.head()


(14874, 21)


,product_id,product_name,sector,main_packaging_material,product_strength_req,weight_capacity_score,barrier_score,biodegradability_score,co2_emission_score,recyclability_percent_x,...,reuse_potential_score,material_id,material_name,material_strength,weight_cap,biodegradability,co2_emission,recyclability_percent_y,strength_gap,weight_gap
0,1,Smartphone box,Electronics,recycled cardboard box + molded pulp tray,3,3,3,85,70,72,...,2,1,Recycled Cardboard / Paperboard,3,3,85,70,75,0,0
1,1,Smartphone box,Electronics,recycled cardboard box + molded pulp tray,3,3,3,85,70,72,...,2,2,Molded Pulp (Recycled Paper Fiber),3,3,85,70,70,0,0
2,1,Smartphone box,Electronics,recycled cardboard box + molded pulp tray,3,3,3,85,70,72,...,2,3,"Kraft Paper Wraps, Paper Padding",2,2,90,70,70,-1,-1
3,1,Smartphone box,Electronics,recycled cardboard box + molded pulp tray,3,3,3,85,70,72,...,2,4,Starch-Based Foams/Peanuts,2,3,95,85,30,-1,0
4,1,Smartphone box,Electronics,recycled cardboard box + molded pulp tray,3,3,3,85,70,72,...,2,5,Mycelium-Based Molded Packaging,3,3,95,90,20,0,0


In [23]:
merged.to_csv("final_ml_dataset.csv", index=False)
